<a href="https://colab.research.google.com/github/borodark/Goodrich/blob/master/Assignment4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
![ ! -e "$(basename spark-2.4.1-bin-hadoop2.7.tgz)" ] && wget  http://apache.osuosl.org/spark/spark-2.4.1/spark-2.4.1-bin-hadoop2.7.tgz
!tar xf spark-2.4.1-bin-hadoop2.7.tgz

--2019-04-09 00:58:38--  http://apache.osuosl.org/spark/spark-2.4.1/spark-2.4.1-bin-hadoop2.7.tgz
Resolving apache.osuosl.org (apache.osuosl.org)... 64.50.236.52, 140.211.166.134, 64.50.233.100, ...
Connecting to apache.osuosl.org (apache.osuosl.org)|64.50.236.52|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 230778742 (220M) [application/x-gzip]
Saving to: ‘spark-2.4.1-bin-hadoop2.7.tgz’

spark-2.4.1-bin-had 100%[===================>] 220.09M  26.1MB/s    in 13s     

2019-04-09 00:58:51 (17.0 MB/s) - ‘spark-2.4.1-bin-hadoop2.7.tgz’ saved [230778742/230778742]



In [0]:
!pip install -q findspark

In [0]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.1-bin-hadoop2.7"

In [0]:
import findspark
findspark.init()
from pyspark.sql import SparkSession

# get a spark session. 
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [64]:

ss_ = spark.read.csv("ss.csv", header= True, 
                      inferSchema = True)

print(ss_.columns)

columns_lambda = lambda k: k.endswith(', Current week')

common_cols = ['Reporting Area','MMWR Year','MMWR Week']

sss = filter(columns_lambda, ss_.columns)


to_keep = common_cols + list(sss)

dfss = ss_.select(*to_keep).withColumnRenamed('Reporting Area', 'area').withColumnRenamed('MMWR Year','year').withColumnRenamed('MMWR Week','week')
dfss.columns

['Reporting Area', 'MMWR Year', 'MMWR Week', 'Salmonellosis (excluding Paratyphoid fever andTyphoid fever)†, Current week', 'Salmonellosis (excluding Paratyphoid fever andTyphoid fever)†, Current week, flag', 'Salmonellosis (excluding Paratyphoid fever andTyphoid fever)†, Previous 52 weeks Med', 'Salmonellosis (excluding Paratyphoid fever andTyphoid fever)†, Previous 52 weeks Med, flag', 'Salmonellosis (excluding Paratyphoid fever andTyphoid fever)†, Previous 52 weeks Max', 'Salmonellosis (excluding Paratyphoid fever andTyphoid fever)†, Previous 52 weeks Max, flag', 'Salmonellosis (excluding Paratyphoid fever andTyphoid fever)†, Cum 2018', 'Salmonellosis (excluding Paratyphoid fever andTyphoid fever)†, Cum 2018, flag', 'Salmonellosis (excluding Paratyphoid fever andTyphoid fever)†, Cum 2017', 'Salmonellosis (excluding Paratyphoid fever andTyphoid fever)†, Cum 2017, flag', 'Shiga toxin-producing Escherichia coli, Current week', 'Shiga toxin-producing Escherichia coli, Current week, flag

['area',
 'year',
 'week',
 'Salmonellosis (excluding Paratyphoid fever andTyphoid fever)†, Current week',
 'Shiga toxin-producing Escherichia coli, Current week',
 'Shigellosis, Current week']

In [68]:
ch_ = spark.read.csv("ch.csv", header= True, 
                      inferSchema = True)

sss = filter(columns_lambda, ch_.columns)


to_keep = common_cols + list(sss)

chss = ch_.select(*to_keep).withColumnRenamed('Reporting Area', 'area').withColumnRenamed('MMWR Year','year').withColumnRenamed('MMWR Week','week')

chss.columns

['area',
 'year',
 'week',
 'Chlamydia trachomatis infection, Current week',
 'Coccidioidomycosis, Current week']

In [83]:
to_keep = common_cols + list(sss)

cghss = chss.join(dfss, [chss.area == dfss.area, chss.week == dfss.week ,chss.year == dfss.year] )
cghss.show(100)

+--------------+----+----+---------------------------------------------+--------------------------------+--------------+----+----+---------------------------------------------------------------------------+----------------------------------------------------+-------------------------+
|          area|year|week|Chlamydia trachomatis infection, Current week|Coccidioidomycosis, Current week|          area|year|week|Salmonellosis (excluding Paratyphoid fever andTyphoid fever)†, Current week|Shiga toxin-producing Escherichia coli, Current week|Shigellosis, Current week|
+--------------+----+----+---------------------------------------------+--------------------------------+--------------+----+----+---------------------------------------------------------------------------+----------------------------------------------------+-------------------------+
| UNITED STATES|2018|   1|                                         3741|                             262| UNITED STATES|2018|   1|            